In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### Wide Residual Network (WRN)

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00 \
    scheduler.epochs 200

[2020-06-12 01:08:42] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: 

In [7]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/checkpoint_00200.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/test_results_0200

[2020-06-12 23:27:03] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/checkpoint_00200.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:50<00:00,  1.27s/it]
[2020-06-12 23:27:55] __main__ INFO: Elapsed 51.00
[2020-06-12 23:27:55] __main__ INFO: Loss 0.1760 Accuracy 0.9578


In [6]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/checkpoint_00100.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/test_results_0100

[2020-06-12 23:25:26] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/checkpoint_00100.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 40/40 [00:50<00:00,  1.27s/it]
[2020-06-12 23:26:18] __main__ INFO: Elapsed 50.91
[2020-06-12 23:26:18] __main__ INFO: Loss 0.2299 Accuracy 0.9311


In [8]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['wrn_28_10', 'wrn_28_10'],
           'Testset': ['cifar10', 'cifar10'],
           'Epoch': [100, 200],
           'Loss': [0.2299, 0.1760],
           'Accuracy': [0.9311, 0.9578],
           'Original_Accuracy': [95.9, 95.9],
           'Original_CI': [(95.5, 96.3), (95.5, 96.3)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])

df.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10,cifar10,100,0.2299,0.9311,95.9,"(95.5, 96.3)"
1,wrn_28_10,cifar10,200,0.1760,0.9578,95.9,"(95.5, 96.3)"


In [9]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/test_results_0200/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-1.3674926 , -1.6152366 , -0.44728577, ..., -1.0346686 ,
        -1.4118685 , -1.5809278 ],
       [-0.10573413,  0.7069439 , -1.0454859 , ..., -1.8201877 ,
         9.020155  , -1.1099735 ],
       [-0.573276  ,  3.92051   , -1.4367174 , ..., -1.7010452 ,
         6.67665   , -0.6017765 ],
       ...,
       [-1.6388117 , -1.4345514 , -0.49067357, ..., -0.44160566,
        -1.5362647 , -1.6407192 ],
       [-0.81282467,  8.434932  , -0.7057322 , ..., -1.4298878 ,
        -0.2568018 , -0.4748483 ],
       [-1.2681558 , -1.3762753 , -1.0645399 , ...,  8.957951  ,
        -1.9260851 , -0.6347342 ]], dtype=float32)

In [10]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/wrn_28_10'
path = '/home/ec2-user/SageMaker/experiments/wrn_28_10'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)

In [ ]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/checkpoint_00200.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/test_results_0200_CIFAR101